# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 9:12AM,258019,10,SONSB0002113,"Nextsource Biotechnology, LLC",Released
1,Feb 23 2023 9:11AM,258018,10,SOTAH0000917,Ivaoes Animal Health,Released
2,Feb 23 2023 8:54AM,258017,12,U1226,"Uniderm USA, Inc",Released
3,Feb 23 2023 8:51AM,258012,12,HH-40117,Hush Hush,Released
4,Feb 23 2023 8:51AM,258012,12,HH-40118,Hush Hush,Released
5,Feb 23 2023 8:51AM,258012,12,HH-40119,Hush Hush,Released
6,Feb 23 2023 8:51AM,258012,12,HH-40120,Hush Hush,Released
7,Feb 23 2023 8:51AM,258012,12,HH-40121,Hush Hush,Released
8,Feb 23 2023 8:51AM,258012,12,HH-40122,Hush Hush,Released
9,Feb 23 2023 8:51AM,258012,12,HH-40111,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,258015,Released,1
34,258016,Released,1
35,258017,Released,1
36,258018,Released,1
37,258019,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258015,NaN,NaN,1.0
258016,NaN,NaN,1.0
258017,NaN,NaN,1.0
258018,NaN,NaN,1.0
258019,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257914,14.0,2.0,3.0
257934,0.0,0.0,2.0
257935,2.0,1.0,14.0
257945,0.0,1.0,0.0
257946,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257914,14,2,3
257934,0,0,2
257935,2,1,14
257945,0,1,0
257946,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257914,14,2,3
1,257934,0,0,2
2,257935,2,1,14
3,257945,0,1,0
4,257946,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257914,14,2,3
1,257934,,,2
2,257935,2,1,14
3,257945,,1,
4,257946,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC"
1,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health
2,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc"
3,Feb 23 2023 8:51AM,258012,12,Hush Hush
22,Feb 23 2023 8:50AM,258016,12,Hush Hush
23,Feb 23 2023 8:50AM,258014,10,ISDIN Corporation
41,Feb 23 2023 8:50AM,258015,12,Hush Hush
42,Feb 23 2023 8:47AM,258013,10,"Senseonics, Incorporated"
43,Feb 23 2023 8:37AM,258010,16,Innova Softgel LLC
44,Feb 23 2023 8:36AM,258009,18,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",,,1
1,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,,,1
2,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",,,1
3,Feb 23 2023 8:51AM,258012,12,Hush Hush,,,19
4,Feb 23 2023 8:50AM,258016,12,Hush Hush,,,1
5,Feb 23 2023 8:50AM,258014,10,ISDIN Corporation,,,18
6,Feb 23 2023 8:50AM,258015,12,Hush Hush,,,1
7,Feb 23 2023 8:47AM,258013,10,"Senseonics, Incorporated",,,1
8,Feb 23 2023 8:37AM,258010,16,Innova Softgel LLC,,,1
9,Feb 23 2023 8:36AM,258009,18,Innova Softgel LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",1,,
1,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,1,,
2,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",1,,
3,Feb 23 2023 8:51AM,258012,12,Hush Hush,19,,
4,Feb 23 2023 8:50AM,258016,12,Hush Hush,1,,
5,Feb 23 2023 8:50AM,258014,10,ISDIN Corporation,18,,
6,Feb 23 2023 8:50AM,258015,12,Hush Hush,1,,
7,Feb 23 2023 8:47AM,258013,10,"Senseonics, Incorporated",1,,
8,Feb 23 2023 8:37AM,258010,16,Innova Softgel LLC,1,,
9,Feb 23 2023 8:36AM,258009,18,Innova Softgel LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",1,,
1,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,1,,
2,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",1,,
3,Feb 23 2023 8:51AM,258012,12,Hush Hush,19,,
4,Feb 23 2023 8:50AM,258016,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,1.0,NaN,NaN
2,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",1.0,NaN,NaN
3,Feb 23 2023 8:51AM,258012,12,Hush Hush,19.0,NaN,NaN
4,Feb 23 2023 8:50AM,258016,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:12AM,258019,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Feb 23 2023 9:11AM,258018,10,Ivaoes Animal Health,1.0,0.0,0.0
2,Feb 23 2023 8:54AM,258017,12,"Uniderm USA, Inc",1.0,0.0,0.0
3,Feb 23 2023 8:51AM,258012,12,Hush Hush,19.0,0.0,0.0
4,Feb 23 2023 8:50AM,258016,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5159566,227.0,7.0,34.0
12,1032060,22.0,0.0,0.0
16,258010,1.0,0.0,0.0
18,258009,1.0,0.0,0.0
19,258001,24.0,12.0,0.0
21,773891,2.0,1.0,0.0
22,773997,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5159566,227.0,7.0,34.0
1,12,1032060,22.0,0.0,0.0
2,16,258010,1.0,0.0,0.0
3,18,258009,1.0,0.0,0.0
4,19,258001,24.0,12.0,0.0
5,21,773891,2.0,1.0,0.0
6,22,773997,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,227.0,7.0,34.0
1,12,22.0,0.0,0.0
2,16,1.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,24.0,12.0,0.0
5,21,2.0,1.0,0.0
6,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,227.0
1,12,Released,22.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,21,22
Status,,,,,,,
Completed,34.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,7.0,0.0,0.0,0.0,12.0,1.0,1.0
Released,227.0,22.0,1.0,1.0,24.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,21,22
0,Completed,34.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,0.0,0.0,12.0,1.0,1.0
2,Released,227.0,22.0,1.0,1.0,24.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,21,22
0,Completed,34.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,0.0,0.0,12.0,1.0,1.0
2,Released,227.0,22.0,1.0,1.0,24.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()